# LangChain RAG 201

Tentative techniques to explore:

1. MultiQueryRetriever
2. Contextual Compression
3. Ensemble Retrievers
4. Self-quering Retrievers
5. Time weighted vector store retrievers

In [1]:
%pip install --q langchain
%pip install --q langchain-community

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# %> curl -fsSL https://ollama.com/install.sh | sh
# %> ollama serve
# %> ollama pull llama2
# %> ollama pull nomic-embed-text

!ollama list

NAME         	ID          	SIZE  	MODIFIED       
llama2:latest	78e26419b446	3.8 GB	17 minutes ago	


In [2]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=0.9,
)

llm("Tell me 5 facts about Roman history")

/usr/local/lib/python3.9/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Of course! Here are five interesting facts about Roman history:

1. The Roman Empire was the first global superpower: At its peak, the Roman Empire stretched from Britain to Egypt and from Spain to Syria, covering an area of approximately 5.9 million square miles. This makes it the first global superpower in human history.
2. Rome was originally a kingdom, not an empire: The city of Rome was founded in 753 BC by Romulus, who was said to have been raised by a she-wolf. The early Roman Kingdom was ruled by kings, but it wasn't until the late Republic period (around 100 BC) that the term "empire" began to be used to describe Rome's growing territorial reach.
3. Julius Caesar was assassinated on the Ides of March: On March 15, 44 BC, a group of senators led by Marcus Junius Brutus and Gaius Cassius Longinus stabbed Julius Caesar to death in the Roman Senate. This event is now known as the Ides of March and is still commemorated today.
4. The Roman Colosseum was originally a gift from the e

'Of course! Here are five interesting facts about Roman history:\n\n1. The Roman Empire was the first global superpower: At its peak, the Roman Empire stretched from Britain to Egypt and from Spain to Syria, covering an area of approximately 5.9 million square miles. This makes it the first global superpower in human history.\n2. Rome was originally a kingdom, not an empire: The city of Rome was founded in 753 BC by Romulus, who was said to have been raised by a she-wolf. The early Roman Kingdom was ruled by kings, but it wasn\'t until the late Republic period (around 100 BC) that the term "empire" began to be used to describe Rome\'s growing territorial reach.\n3. Julius Caesar was assassinated on the Ides of March: On March 15, 44 BC, a group of senators led by Marcus Junius Brutus and Gaius Cassius Longinus stabbed Julius Caesar to death in the Roman Senate. This event is now known as the Ides of March and is still commemorated today.\n4. The Roman Colosseum was originally a gift fr

https://www.youtube.com/watch?v=2nLDe_igVnE
https://github.com/langchain-ai/langchain/blob/master/cookbook/nomic_embedding_rag.ipynb


In [3]:
# Optional: LangSmith API keys
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__f40e06d2f1164365aa0ea071877c049c"

In [21]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")
text = "Embed this text"
embed = embeddings.embed_query(text)
len(embed)

768

In [4]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
doc_list = [item for sublist in docs for item in sublist]